<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os
import textwrap

# fix random seed for reproducibility
np.random.seed(45)

Using TensorFlow backend.


## Loading and Preprocessing

In [2]:
url = "https://www.gutenberg.org/files/100/100-0.txt"
path_to_file = tf.keras.utils.get_file('shakespeare.txt', url)
raw_text = open(path_to_file, 'r', encoding='utf-8').read()
print(raw_text[:100])

﻿
Project Gutenberg’s The Complete Works of William Shakespeare, by William
Shakespeare

This eBook 


In [3]:
def pre_process(text):
    text = text[900:-21100]
    text = text.replace("\r", "")
    text = " ".join(text.split())
    print (f'Length of text: {len(text)} characters long')
    return text
text = pre_process(raw_text)
text[:100]

Length of text: 5256461 characters long


'Contents THE SONNETS ALL’S WELL THAT ENDS WELL THE TRAGEDY OF ANTONY AND CLEOPATRA AS YOU LIKE IT TH'

In [4]:
# create mapping of unique chars to integers
chars = sorted(set(text))
char2idx = {c:i for i, c in enumerate(chars)}
idx2char = {i:c for i, c in enumerate(chars)}

n_chars = len(text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Unique Characters: ", n_vocab)

Total Characters:  5256461
Total Unique Characters:  99


We can see that the text blob have 500K characters, and that each converted to lowercase with no `\r` gave us 99 unique characters. 

In [5]:
seq_length = 100

X_data= []
y_data = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    X_data.append([char2idx[char] for char in seq_in])
    y_data.append(char2idx[seq_out])

n_patterns = len(X_data)
print("Total Patterns: ", n_patterns)

Total Patterns:  5256361


In [6]:
# Reshape X to be [samples, time steps, features]
X = np.reshape(X_data, (n_patterns, seq_length, 1))

# Normalize
X = X / float(n_vocab)

# One hot encode the output variable
y = to_categorical(y_data)

print(f'Shape of X: {X.shape}')
print(f'Shape of y: {y.shape}')

Shape of X: (5256361, 100, 1)
Shape of y: (5256361, 99)


### Create Model

In [7]:
def create_model1(X, y):
    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model


model1 = create_model1(X, y)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Add callbacks

In [8]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="angus")

# define the checkpoint
# Directory where the checkpoints will be saved
checkpoint_directory = './training_checkpoints'

if not os.path.exists(checkpoint_directory ):
    os.makedirs(checkpoint_directory)
    print(f'New directory created at $:->{checkpoint_directory}')
file_name ="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint_filename = os.path.join(checkpoint_directory, file_name)
checkpoint_callback = ModelCheckpoint(checkpoint_filename, monitor='loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint_callback, WandbCallback()]

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


### Train/Fit Model

In [ ]:
wandb.config.epochs = 20
wandb.config.batch_size = 128
history = model1.fit(X, y, 
                     validation_split=0.33, 
                     batch_size=wandb.config.batch_size, 
                     epochs=wandb.config.epochs, 
                     callbacks=callbacks_list
                    )

## Plot Loss

In [ ]:
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN